In [1]:
import sqlite3
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import datetime as dt
from urllib.parse import urlparse
import pytz

In [7]:
#locate the browsing history database
history_db_path = r'C:\Users\xjing\AppData\Local\Google\Chrome\User Data\Profile 5\History'

# Create a copy of the database to avoid issues with running Chrome at the same time
temp_history_db_path = r'C:\Users\xjing\AppData\Local\Temp\History_Copy'
shutil.copyfile(history_db_path, temp_history_db_path)

#connect to the SQLite database
conn = sqlite3.connect(temp_history_db_path)
cursor = conn.cursor() 

#sql query for obtaining the url,url title and visit time of browsing histories
query = """
    SELECT 
        visits.id, 
        urls.url, 
        urls.title, 
        visits.visit_time, 
        keyword_search_terms.term AS search_term
    FROM 
        visits
    INNER JOIN 
        urls ON visits.url = urls.id
    LEFT JOIN 
        keyword_search_terms ON visits.url = keyword_search_terms.url_id
"""
#read the obtained data into a dataframe
df = pd.read_sql_query(query, conn)

# convert the visit_time to readable format,in Melbourne timezone
def convert_time(timestamp):
    melbourne_tz = pytz.timezone('Australia/Melbourne')
    dt = datetime.datetime(1601, 1, 1,tzinfo=pytz.UTC) + datetime.timedelta(microseconds=timestamp)
    dt = dt.astimezone(melbourne_tz)
    return dt.replace(microsecond=0)

# apply the function to the 'visit_time' column
df['visit_time'] = df['visit_time'].apply(convert_time)

#close the database connection
conn.close()

#remove temporary copy
os.remove(temp_history_db_path)

In [8]:
#filter by day, display list view
df['visit_date'] = df['visit_time'].dt.date
df['visit_time_minute'] = df['visit_time'].apply(lambda x: x.replace(second = 0))

# Sort by date and time
df.sort_values(['visit_date', 'visit_time'], ascending=[False, False], inplace=True)

# Drop duplicates -> visits to the same url in the same minute
df.drop_duplicates(subset=['visit_time_minute', 'url'], inplace=True, keep = 'last')

# group by 'visit_date', sort descendingly
grouped_by_day = df.groupby('visit_date', sort=False)

# loop through each group and print out the records for that day
for name, group in grouped_by_day:
    print(f"Date:{name}")
    print("="*50)  #add separator between days
    for index, row in group.iterrows():
        print(f"{row['url']},{row['title']},{row['visit_time'].time()}")
    print()

Date:2023-09-06
https://chrome.google.com/webstore/detail/better-history/egehpkpgpgooebopjihjmnpejnjafefi/related,Better History - Chrome Web Store,21:40:29
https://chrome.google.com/webstore/detail/better-history/egehpkpgpgooebopjihjmnpejnjafefi,Better History - Chrome Web Store,21:40:07
https://www.google.com/search?q=browse+history+%2B+extension&sca_esv=563044254&ei=CWX4ZJb_LZyo4-EPpOeW4AQ&ved=0ahUKEwjWp-Cd85WBAxUc1DgGHaSzBUwQ4dUDCBA&uact=5&oq=browse+history+%2B+extension&gs_lp=Egxnd3Mtd2l6LXNlcnAiGmJyb3dzZSBoaXN0b3J5ICsgZXh0ZW5zaW9uMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeMgYQABgWGB4yCBAAGBYYHhgPMggQABgWGB4YDzIIEAAYCBgeGA0yCBAAGIoFGIYDMggQABiKBRiGAzIIEAAYigUYhgNIsRNQnwFYoBFwAngBkAEAmAHpAaAB0g-qAQUwLjcuNLgBA8gBAPgBAcICChAAGEcY1gQYsAPCAggQABgWGB4YCuIDBBgAIEGIBgGQBgg&sclient=gws-wiz-serp&bshm=rime/1,browse history + extension - Google Search,21:40:02
https://www.google.com/search?q=browse+history+%2B&oq=browse+history+%2B&aqs=chrome..69i57j0i22i30j0i10i22i30j0i22i30l7.6586j0j7&sourceid=chrome&